In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


Import the dataset

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/jokes-data.csv')

In [4]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3156233 sha256=45d0ac89c6fefb2b3fe3a88f3095071b6d787686d8cceb302426854c4d59612e
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [5]:
data.head()

,id,user_id,joke_id,Rating
0,31030_110,31030,110,2.750
1,16144_109,16144,109,5.094
2,23098_6,23098,6,-6.438
3,14273_86,14273,86,4.406
4,18419_134,18419,134,9.375


In [6]:
data.isna().sum()

id         0
user_id    0
joke_id    0
Rating     0
dtype: int64

In [7]:
data.dtypes

id          object
user_id      int64
joke_id      int64
Rating     float64
dtype: object

In [8]:
from surprise import Reader, Dataset, KNNBasic, SVD

In [9]:
data.Rating.unique()

array([  2.75 ,   5.094,  -6.438,   4.406,   9.375,  -1.781,   6.25 ,
        -2.906,  -5.75 ,   1.656,   0.688,   1.312,   6.031,   9.781,
         8.75 ,   0.156,   3.562,   2.938,   3.75 ,  -1.906,   3.469,
         7.   ,   4.656,   1.719,   6.219,  -8.25 ,   5.188,  -2.281,
         1.406,   2.906,   3.625,   6.719,   5.656,   8.562,   2.156,
        -5.344,   3.25 ,   1.   ,   7.375,   8.031,   6.125,   7.469,
         2.125,   4.062,  -8.   ,  -2.75 ,   9.406,   8.875,   4.5  ,
        -3.188,  -6.875,   2.719,   0.562,   2.969,   8.375,  -4.906,
         8.594,  -7.469,   0.25 ,   6.406,   5.125,   3.156,   0.312,
         0.969,  -3.969,   5.438,  -0.031,  -1.969,   4.688,   2.781,
         0.875,  -9.625,  -2.188,  -3.062,   8.781,   6.938,  -7.406,
         9.281,  -0.5  ,   1.25 ,   5.031,   3.375,   4.375,   6.875,
         6.688,   8.531,   5.594,   7.75 ,   8.281,  -0.531,   9.5  ,
         6.812,  -4.125,   1.875,   3.688,   4.438,  -5.5  ,  -1.406,
         4.031,  -0.

In [10]:
data.Rating.min()

-10.0

In [11]:
data.Rating.max()

10.0

scaling the ratings into the range of [1,5]

In [12]:
old_min = -10
old_max = 10
new_min = 1
new_max = 5
scaling_factor = (new_max - new_min) / (old_max - old_min)

def rating_converter(original_rating):
  global scaling_factor
  new_rating = (original_rating - old_min) * scaling_factor + new_min
  return new_rating

data['Scaled_Rating'] = data.Rating.apply(rating_converter)

In [13]:
data

,id,user_id,joke_id,Rating,Scaled_Rating
0,31030_110,31030,110,2.750,3.5500
1,16144_109,16144,109,5.094,4.0188
2,23098_6,23098,6,-6.438,1.7124
3,14273_86,14273,86,4.406,3.8812
4,18419_134,18419,134,9.375,4.8750
...,...,...,...,...,...
1092054,9517_132,9517,132,3.156,3.6312
1092055,27767_118,27767,118,-1.594,2.6812
1092056,10580_81,10580,81,2.000,3.4000
1092057,31007_119,31007,119,8.906,4.7812


Dropping the old rating

In [14]:
data.drop('Rating', axis=1, inplace=True)

In [15]:
data

,id,user_id,joke_id,Scaled_Rating
0,31030_110,31030,110,3.5500
1,16144_109,16144,109,4.0188
2,23098_6,23098,6,1.7124
3,14273_86,14273,86,3.8812
4,18419_134,18419,134,4.8750
...,...,...,...,...
1092054,9517_132,9517,132,3.6312
1092055,27767_118,27767,118,2.6812
1092056,10580_81,10580,81,3.4000
1092057,31007_119,31007,119,4.7812


Dropping the id column as it is not necessary

In [16]:
data.drop('id', axis=1, inplace=True)

In [17]:
reader = Reader(rating_scale=(1,5))

In [18]:
surprise_dataset = Dataset.load_from_df(data, reader)

KNN Basic Algorithmn is failling due to the size of data set so skipping it

In [19]:
algo = SVD()

In [20]:
from surprise.model_selection import cross_validate

In [21]:
cross_validate(algo, surprise_dataset, measures=['RMSE'], cv=5)

{'test_rmse': array([0.82207983, 0.82413422, 0.82485176, 0.82515253, 0.82541813]),
 'fit_time': (12.337071657180786,
  13.541559219360352,
  12.343778133392334,
  12.255706071853638,
  12.898530721664429),
 'test_time': (3.9514710903167725,
  1.8558037281036377,
  2.3640053272247314,
  1.810748815536499,
  1.8063685894012451)}

In [22]:
#NMF

In [23]:
from surprise import NMF

In [24]:
algo2 = NMF()

In [25]:
cross_validate(algo2, surprise_dataset, measures=['RMSE'], cv=5)

{'test_rmse': array([0.86177664, 0.85974977, 0.86411825, 0.86465881, 0.85870697]),
 'fit_time': (25.962722539901733,
  23.286277770996094,
  24.206597566604614,
  22.93079161643982,
  22.896637678146362),
 'test_time': (2.6870219707489014,
  1.7808659076690674,
  1.6057629585266113,
  1.5862181186676025,
  2.2553699016571045)}

In [26]:
#SlopOne

In [27]:
from surprise import SlopeOne
algo3 = SlopeOne()

In [28]:
cross_validate(algo3, surprise_dataset, measures=['RMSE'], cv=5)

{'test_rmse': array([0.84191951, 0.84150595, 0.84357277, 0.84135479, 0.84478973]),
 'fit_time': (1.1017389297485352,
  1.3339920043945312,
  1.3415355682373047,
  1.3839712142944336,
  1.3456802368164062),
 'test_time': (7.3513343334198,
  11.326664447784424,
  7.718003034591675,
  6.926027059555054,
  6.779803514480591)}

In [29]:
#Baseline

In [31]:
from surprise import BaselineOnly

In [32]:
algo4 = BaselineOnly()
cross_validate(algo3, surprise_dataset, measures=['RMSE'], cv=5)

{'test_rmse': array([0.84377514, 0.8440418 , 0.8410673 , 0.84363578, 0.84104402]),
 'fit_time': (2.0457675457000732,
  2.043200969696045,
  1.4105212688446045,
  1.3874454498291016,
  1.36574387550354),
 'test_time': (7.469939231872559,
  6.967691898345947,
  8.190925121307373,
  7.6825408935546875,
  8.153337478637695)}

SVD has the lowest RMSE so doing hyperparamter tuning to further improve the model

In [33]:
from surprise.model_selection import GridSearchCV
param_grid = {
    'n_factors': [10, 20, 30],
    'n_epochs': [5, 10, 20],
    'lr_all': [0.002, 0.005, 0.01]
}

Splitting the dataset

In [34]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(surprise_dataset, test_size=0.2)

In [42]:
best_rmse = float('inf')
best_params = {}

for n_factors in param_grid['n_factors']:
    for n_epochs in param_grid['n_epochs']:
        for lr_all in param_grid['lr_all']:
            algo = SVD(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr_all, verbose=False)
            results = cross_validate(algo, surprise_dataset, measures=['RMSE'], cv=5, verbose=False)
            mean_rmse = results['test_rmse'].mean()
            if mean_rmse < best_rmse:
                best_rmse = mean_rmse
                best_params = {'n_factors': n_factors, 'n_epochs': n_epochs, 'lr_all': lr_all}

print("Best Hyperparameters:", best_params)
print("Best RMSE:", best_rmse)


Best Hyperparameters: {'n_factors': 20, 'n_epochs': 20, 'lr_all': 0.005}
Best RMSE: 0.8225264566666208


In [43]:
best_model = SVD(n_factors = 20, n_epochs=20, lr_all=0.005)


In [45]:
best_model.fit(trainset)

In [46]:
predictions = best_model.test(testset)

In [47]:
from surprise import accuracy
rmse = accuracy.rmse(predictions)

RMSE: 0.8225
